In [1]:
%classpath add mvn org.apache.spark spark-sql_2.11 2.4.4


In [ ]:
%classpath add mvn org.apache.spark spark-mllib_2.11 2.4.4


In [3]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
                        .appName("Oscar Prediction App")
                        .master("local[2]")
                        .config("spark.ui.enabled", "false")
                        .getOrCreate()
val sc = spark.sparkContext

org.apache.spark.SparkContext@7d61ffc1

In [4]:
val oscarDF = spark.read
  .option("header", "true")
  .option("inferSchema", "true")
  .csv("data/movie_data.csv")  

[Year: int, Movie: string ... 59 more fields]

In [5]:
oscarDF.printSchema


root
 |-- Year: integer (nullable = true)
 |-- Movie: string (nullable = true)
 |-- Oscar_winner: integer (nullable = true)
 |-- Oscar_nominee: integer (nullable = true)
 |-- Runtime (min): integer (nullable = true)
 |-- Certificate: string (nullable = true)
 |-- Directors: string (nullable = true)
 |-- Actors: string (nullable = true)
 |-- Metascore: string (nullable = true)
 |-- IMDb_rating: string (nullable = true)
 |-- IMDb_votes: string (nullable = true)
 |-- RT_rating: double (nullable = true)
 |-- RT_review: double (nullable = true)
 |-- Budget: integer (nullable = true)
 |-- Domestic (US) gross: integer (nullable = true)
 |-- International gross: integer (nullable = true)
 |-- Worldwide gross: long (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Action: string (nullable = true)
 |-- Adventure: string (nullable = true)
 |-- Animation: string (nullable = true)
 |-- Biography: integer (nullable = true)
 |-- Comedy: integer (nullable = true)
 |-- Crime: integer (nullabl

[Oscar_winner: int, Genre: string ... 1 more field]

[[List(0, Sci-Fi,Thriller,, 2), List(0, Action,Drama,Sport, 3), List(0, Drama,History,Music, 1), List(0, Mystery,Thriller,, 9), List(0, Comedy,Drama,Family, 4), List(0, Action,Drama,Family, 2), List(0, Animation,Comedy,Fantasy, 1), List(0, Biography,Drama,Music, 6), List(0, Drama,Fantasy,Mystery, 3), List(0, Action,Adventure,Comedy, 56), List(0, Drama,Romance,Sci-Fi, 7), List(0, 80147166, 1), List(0, Action,Drama,Sci-Fi, 9), List(0, Crime,Mystery,Thriller, 8), List(0, Crime,Drama,Western, 1), List(0, Crime,Drama,Fantasy, 1), List(0, Drama,History,Thriller, 8), List(0, Crime,Drama,Musical, 1), List(0, Drama,Fantasy,Sport, 1), List(0, Action,Crime,, 2), List(0, Comedy,Drama,Fantasy, 14), List(0, Drama,Fantasy,Horror, 12), List(0, Action,Drama,History, 12), List(0, 102427862, 1), List(0, Action,Horror,Thriller, 5), List(1, Adventure,Drama,Fantasy, 2), List(0, Comedy,Drama,Thriller, 1), List(0, Adventure,Comedy,Music, 1), List(0, Comedy,Drama,Musical, 4), List(0, 90376224, 1), List(0, 1346

In [ ]:
import org.apache.spark.sql.functions.udf
import spark.implicits._ 
val genre=(genres:String)=>{
 genres.split(",")(0)}

val uniqueGenre=udf(genre)

val oscarDF1= oscarDF
        .withColumn("FilmGenre", uniqueGenre($"Genre"))
        .drop("Genre")

oscarDF1.show

+----+--------------------+------------+-------------+-------------+-----------+--------------------+--------------------+---------+-----------+----------+---------+---------+---------+-------------------+-------------------+---------------+------+---------+---------+---------+------+-----+-----+------+-------+-------+------+-------+-------+-------+------+-----+--------+---+-------+------------+-------------+----------+-----------+----------------+-----------------+---------------+----------------+----------+-----------+-----------+------------+------------------+-------------------+------------------+-------------------+------------------+-------------------+----------+-----------+------------+-------------+-----------+------------+---------+
|Year|               Movie|Oscar_winner|Oscar_nominee|Runtime (min)|Certificate|           Directors|              Actors|Metascore|IMDb_rating|IMDb_votes|RT_rating|RT_review|   Budget|Domestic (US) gross|International gross|Worldwide gross|Actio

|1999|Austin Powers: Th...|           0|            0|           95|         PG|           Jay Roach|['Mike Myers', 'H...|       59|        6.6|    210376|     52.0|     87.0| 35000000|          206040085|          106343402|      312383487|     1|        1|        0|        0|     1|    0|    0|     0|      0|      0|     0|      0|      0|      0|     0|    0|       0|  0|      0|           0|            0|         0|          0|               0|                0|              0|               0|         0|          0|          0|           0|                 0|                  0|                 0|                  0|                 0|                  0|         0|          0|           0|            0|          0|           0|   Action|
|1999|               Dogma|           0|            0|          130|   (Banned)|         Kevin Smith|['Ben Affleck', '...|       62|        7.3|    203357|     67.0|    127.0| 10000000|           30651422|           13297443|       43948865|     

org.apache.spark.sql.SparkSession$implicits$@547238f

In [27]:
val grByGenreAndWin = oscarDF1.groupBy("Oscar_winner", "FilmGenre").count

[Oscar_winner: int, FilmGenre: string ... 1 more field]

In [28]:
beakerx.grByGenreAndWin =  grByGenreAndWin.collect.map(_.toSeq.toArray)

[[List(0, Drama, 325), List(1, Drama, 5), List(0, 80147166, 1), List(1, Comedy, 4), List(0, Family, 1), List(0, 102427862, 1), List(1, Adventure, 2), List(0, 90376224, 1), List(0, 1346913161, 1), List(0, 4490134, 1), List(1, Action, 1), List(0, 56797392, 1), List(0, 85911226, 1), List(0, Biography, 114), List(0, Horror, 97), List(0, Comedy, 423), List(0, 42587643, 1), List(0, Sci-Fi, 2), List(0, Mystery, 10), List(0, 115637895, 1), List(0, 145026640, 1), List(0, Fantasy, 8), List(0, 80631, 1), List(0, 14558937, 1), List(1, Biography, 6), List(0, 248757044, 1), List(0, Action, 686), List(0, 45863000, 1), List(0, Romance, 1), List(0, 16083005, 1), List(0, 76665507, 1), List(0, 33047633, 1), List(0, 81697192, 1), List(0, 20695413, 1), List(0, 161197785, 1), List(0, 58657005, 1), List(0, 24148068, 1), List(0, 11599903, 1), List(0, 39568572, 1), List(0, Crime, 125), List(0, 13472307, 1), List(0, 10561196, 1), List(0, 81578, 1), List(0, 175084580, 1), List(0, Adventure, 117), List(0, 6948874